# Init hosted instance

In [ ]:
!lamin login testuser1

In [ ]:
import lamindb_setup as ln_setup
from lamindb_setup.core.upath import create_path, InstanceNotEmpty
from lamindb_setup.core._hub_core import (
    delete_instance,
    call_with_fallback_auth,
    select_instance_by_owner_name,
)
from lnschema_core.ids import base62

suffix = base62(8)

instance_name = f"my-hosted-custom-storage-{suffix}"
assert ln_setup.settings.user.handle == "testuser1"

In [ ]:
try:
    delete_instance(f"testuser1/{instance_name}")
except InstanceNotEmpty:
    instance_with_storage = call_with_fallback_auth(
        select_instance_by_owner_name,
        owner="testuser1",
        name=instance_name,
    )
    root = create_path(instance_with_storage["storage"]["root"])
    for obj in root.rglob(""):
        if obj.is_file():
            obj.unlink()
    delete_instance(f"testuser1/{instance_name}")

In [ ]:
ln_setup.init(name=instance_name, storage=f"s3://lamindb-setup-init-test/{suffix}")

In [ ]:
assert ln_setup.settings.instance.storage.type_is_cloud == True
assert ln_setup.settings.instance.owner == ln_setup.settings.user.handle
assert ln_setup.settings.instance.name == instance_name
assert ln_setup.settings.storage.id is not None

assert ln_setup.settings.storage._mark_storage_root.exists()

In [ ]:
ln_setup.close()